# Taller ICMP 2022

Para el siguiente taller utilizaremos `Scapy` para construir, enviar y recibir paquetes. Luego, a partir de los mensajes del protocolo ICMP, implementaremos una herramienta que nos permita seguir por cuales nodos IP pasa un paquete hasta llegar (si llega) a destino.

Los ejercicios los guiarán por diferentes consignas para que luego puedan entender el código necesario del TP. Debajo de cada ejercicio está la respuesta, **recomendamos fuertemente intentar hacer el ejercicio antes de ver la respuesta**.

In [ ]:
import sys
from scapy.all import *
from time import *

## Ejercicio 1: Entrada en calor

Vamos a recordar algunas cosas que vimos en el taller 1. 

`Scapy` es una herramienta útil para el análisis de redes, capturar, enviar y recibir paquetes. Existen distintas funciones que nos pueden ser útiles a la hora de programar un script con Scapy, algunas de estas son:

 * `ls(PROTOCOLO)`: Si hacemos ls(Ether) nos mostrará los distintos campos de los frames Ethers.
 * `lsc()`: Nos devuelve las distintas funciones de Scapy
 * Documentación: https://scapy.readthedocs.io/en/latest/usage.html
 

1. Crear un paquete Ether broadcast.
2. Encapsular un paquete IP dentro de un paquete Ether. La dirección destino del paquete IP tiene que ser la IP de una página web. (Hint: Qué pasa si pongo al dirección web o URL?)

In [ ]:
Ether().show()
Ether()/IP(dst='www.google.com')

## Ejercicio 2: Enviar paquetes

Scapy no sólo permite hacer sniffing, si no que permite enviar paquetes a la red.
Para esto nos provee dos funciones `send()` y `sendp()`. Ambas se utilizan para enviar mensajes, la diferencia es que el primero va por capa 3 (Red) mientras que el segundo por Capa 2 (Medios compartidos).

*Si no hay respuesta el valor será None*


1. Crear un paquete IP con destino a alguna universidad del mundo. 
2. Enviar el paquete de 2.

In [ ]:
p = IP(src="ox.ac.uk")
send(p)

## Ejercicio 3: Enviar y recibir paquetes

Scapy permite enviar paquetes a la red y obtener la respuesta. Para esto nos provee dos funciones `sr()` y `sr1()`. Ambas se utilizan para enviar mensajes y recibir respuestas. La diferencia de `sr1` es que retorna *un* solo paquete. 

Para poder enviar y recibir paquetes `sr()` requiere que los paquetes sean de Capa 3 (e.g. IP) mientras que si necesitamos enviar y recibir por Capa 2 podemos usar `srp()`

*Si no hay respuesta el valor será None*

1. Crear un paquete ICMP con destino a alguna universidad del mundo (hint: ICMP se encapsula con IP).
2. Enviar el paquete de 2. e imprimir la respuesta.

In [ ]:
p=IP(dst='ox.ac.uk')
p2=sr1(IP(dst="ox.ac.uk")/ICMP())
p2.show()

## ICMP: Echo Request (type 8) y Echo Reply (type 0)

Estos mensajes son utilizados para el test de diagnóstico más básico de ICMP, el **ping**. Este test se utiliza conocer si un host es alcanzable. Sin embargo, esta herramienta no devuelve suficiente información. No todos los dominios devuelven una respuesta, por distintos motivos que veremos más adelante.

## Ejercicio 4: Implementar la herramienta PING

1. Crear un paquete ICMP con type=8.
2. Enviar (sr()) a alguna universidad. Además agregar como parámetro timeout=1 en caso de que la universidad no responda.
3. Completar la función para que dado una URL escrita como string, mande un mensaje al dominio y devuelva la respuesta.

In [ ]:
def ping(url):
    p = IP(dst=url)/ICMP(type=8)
    r = sr(p, timeout=1)
    print(r)
ping('www.google.com')

## Destination Unreachable (type 3)

En el caso donde hayan 3 routers A--B--C, el router A manda un mensaje a C, pero al llegar al router B, este no encuentra la ruta correcta al router C. Cómo se enterará A de que esto sucedió? Es eficiente que A siga mandando a ciegas todo el tiempo?

Para esto, se implementa el mensaje ICMP de Destination Unreachable. Este encapsula una parte del datagrama IP que no se entregó. 

**No hay garantías que este mensaje se responda siempre**

Tiene varios subtipos:
* Destination network unreachable (code=0): Si el router no sabe cómo rutear el paquete (e.g. router B no tiene una ruta programada para la red destino)
* Destination host unreachable (code=1): El host destino está en la red del router pero este determinó que no puede llegar al host.
* Destination port unreachable (code=3): El mensaje llegó a destino pero el puerto no tiene un proceso asociado, en este caso es el host destino quien responde.

## Ejercicio 5: Conseguir un destination unreachable.


1. Crear un paquete ICMP cuya respuesta sea Destination Unreachable con subtipo Destination Network Unreachable.
2. Intentar con diferentes IPs conseguir un Destination Unreachable, esto puede ser difícil, si después de varios intentos no sale pasar al siguiente ejercicio.

In [ ]:
p = ICMP(type=3, code=0)
p.show()
a = sr1(IP(dst="192.168.0.7/32")/ICMP(type=8), timeout=1)
print(a)

## Time exceeded (type=11)

* Si se genera un loop, un datagrama IP podría ciclar infinitamente por la red y generar tráfico. 
* Una posible solución es agregar un campo al datagrama denominado **T**ime-**t**o-**l**ive o TTL.
* Cada host que recibe un datagrama IP, si este no es para él, decrementará un valor al TTL y lo enviará al próximo salto, si llega a 0, lo descarta.

Por lo tanto, otros de los tipos de paquetes del protocolo ICMP es el "Time exceeded" e informa cuándo el TTL del datagrama IP llega a 0.

## Ejercicio 6: Generar un time exceeded.

1. Crear el paquete ICMP cuyo tipo sea el de time exceeded.
2. Crear un paquete IP con destino a una universidad agregándole el parámetro ttl que recibe un valor numérico.
3. Enviar el paquete del ejercicio 2 con distintos TTLs para conseguir un time exceeded.
4. Correr el comando `traceroute()` con parámetro una URL.

In [ ]:
p1 = ICMP(type=11)
p2 = IP(dst='ox.ac.uk', ttl=1)/ICMP()
sr1(p2, timeout=1)

def try_ttl():
    for i in range(0,15):
        p2 = IP(dst='ox.ac.uk', ttl=i)/ICMP()
        print(p2[ICMP].type)
        
try_ttl()

## Ejercicio 7: Implementar traceroute

En el siguiente ejercicio construiremos una herramienta utilizada para el diagnóstico de red denominada **traceroute**. Esta herramienta permite conocer la ruta o dispositivos por los que "salta" (hop) un datagrama IP antes de llegar a destino (si es que llega). Existen distintas implementaciones, incluso, cada SO cuenta con la suya. Scapy por supuesto, también. 

Aprovechando lo que hicieron en los ejercicios anteriores, completar el código de abajo para que haga lo siguiente:

1. Dado una URL destino (e.g. dc.uba.ar):

    a) Calcular los RTTs entre cada salto que responda **Time Exceeded**.
    
    b) Mandar al menos 30 paquetes por cada TTL y por cada respuesta promediar el RTT de la IP que más responde.
    
    c) Calcular el RTT entre saltos restando los valores de RTT de saltos sucesivos.
            i. Si la diferencia da negativa se puede obviar este calculo y calcularlo con el próximo salto que de positivo.

In [ ]:
#!/usr/bin/env python3

import sys
from scapy.all import *
from time import *

def traceroute(url):
    responses = {}
    for i in range(2):
        print()
        for ttl in range(1,25):
            probe = IP(dst=url, ttl=ttl) / ICMP()
            t_i = time()
            ans = sr1(probe, verbose=False, timeout=0.8)
            t_f = time()
            rtt = (t_f - t_i)*1000
            if ans is not None:

                if ttl not in responses: 
                    responses[ttl] = []
                responses[ttl].append((ans.src, rtt))

                if ttl in responses: 
                    print(ttl, responses[ttl])